In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from datetime import datetime, timezone
from time import sleep
import requests
import pandas as pd
from torch_geometric.data import Data
from typing import List,Dict,Tuple,Optional

# ----- Spatial encoder: Graph Convolution over trade network -----
class GCNEncoder(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, dropout=0.1):
        super().__init__()
        self.conv1 = GCNConv(in_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, out_dim)
        self.dropout = dropout

    def forward(self, x, edge_index, edge_weight=None):
        # x: [N_nodes, in_dim]
        h = self.conv1(x, edge_index, edge_weight)  # neighbors -> node
        h = F.relu(h)
        h = F.dropout(h, p=self.dropout, training=self.training)
        h = self.conv2(h, edge_index, edge_weight)  # second hop (indirect effects)
        h = F.relu(h)
        return h  # [N_nodes, out_dim]


# ----- Full spatio-temporal model for India -----
class SanctionImpactGNN(nn.Module):
    """
    Spatio-temporal GNN explicitly aligned with literature review channels.

    Spatial:  GCN over country trade graph per year (GDP, CPI, FX, Trade, FDI, etc.)
    Temporal: GRU over India's embedding across years (sanction dynamics)
    Heads:    Predict impact on GDP, CPI, FX, Trade, FDI, Fiscal, Reserves,
              composite ImpactScore_1y, and impact duration.
    """
    def __init__(
        self,
        in_dim,              # node feature dim (matches channels + sanction/NLP features)
        gcn_hidden=64,
        gcn_out=64,
        gru_hidden=64,
        gru_layers=1,
        india_index=0,
        dropout=0.1
    ):
        super().__init__()
        self.india_index = india_index

        # 1. Spatial GCN
        self.gcn = GCNEncoder(in_dim, gcn_hidden, gcn_out, dropout=dropout)

        # 2. Temporal GRU over India's embeddings
        self.gru = nn.GRU(
            input_size=gcn_out,
            hidden_size=gru_hidden,
            num_layers=gru_layers,
            batch_first=True
        )

        # 3. Channel-specific heads (each literature channel gets its own small regressor)
        self.heads = nn.ModuleDict({
            "gdp": nn.Linear(gru_hidden, 1),
            "cpi": nn.Linear(gru_hidden, 1),
            "fx": nn.Linear(gru_hidden, 1),
            "trade": nn.Linear(gru_hidden, 1),
            "fdi": nn.Linear(gru_hidden, 1),
            "res": nn.Linear(gru_hidden, 1),
            "score": nn.Linear(gru_hidden, 1),
            "duration": nn.Linear(gru_hidden, 1),
        })

        self.dropout = nn.Dropout(dropout)

    def _forward_sequence(self, data_list: List) -> Dict[str, torch.Tensor]:
        india_embs = []

        for data in data_list:
            edge_weight = getattr(data, "edge_weight", None)
            h = self.gcn(data.x, data.edge_index, edge_weight)
            india_embs.append(h[self.india_index])

        # [T, gcn_out]
        india_seq = torch.stack(india_embs, dim=0).unsqueeze(0)

        gru_out, _ = self.gru(india_seq)
        h_t = self.dropout(gru_out.squeeze(0))  # [T, gru_hidden]

        return {k: head(h_t) for k, head in self.heads.items()}
    
    #Main forward    
    def forward(self, inputs):
        """
        inputs:
            - List[Data]              → single time sequence
            - List[List[Data]]        → batch of sequences
        """
        if not inputs:
            raise ValueError("Input to SanctionImpactGNN.forward() cannot be empty")
    
        # SINGLE sequence: List[Data]
        if isinstance(inputs[0], Data):
            return self._forward_sequence(inputs)
    
        # BATCH: List[List[Data]]
        batch_preds = {}
        for seq in inputs:
            preds = self._forward_sequence(seq)
            for k, v in preds.items():
                batch_preds.setdefault(k, []).append(v)
    
        for k in batch_preds:
            batch_preds[k] = torch.stack(batch_preds[k], dim=0)  # [B, T, 1]
    
        return batch_preds

C:\Users\KIIT0001\anaconda3\Lib\site-packages\torch_geometric\__init__.py:4: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: Could not load this library: C:\Users\KIIT0001\anaconda3\Lib\site-packages\torch_scatter\_version_cpu.pyd
  import torch_geometric.typing
C:\Users\KIIT0001\anaconda3\Lib\site-packages\torch_geometric\__init__.py:4: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: Could not load this library: C:\Users\KIIT0001\anaconda3\Lib\site-packages\torch_cluster\_version_cpu.pyd
  import torch_geometric.typing
C:\Users\KIIT0001\anaconda3\Lib\site-packages\torch_geometric\__init__.py:4: UserWarning: An issue occurred while importing 'torch-spline-conv'. Disabling its usage. Stacktrace: Could not load this library: C:\Users\KIIT0001\anaconda3\Lib\site-packages\torch_spline_conv\_version_cpu.pyd
  import torch_geometric.typing
C:\Users\KIIT0001\anaconda3\Lib\site-packages\torch_geom

In [2]:
# Sanction type labels (single-label)
sanction_type_map = {
    "trade": 0,
    "financial": 1,
    "arms": 2,
    "travel": 3,
    "other": 4
}

# Economic channel labels (multi-label)
economic_channels = [
    "GDP",
    "Inflation",
    "ExchangeRate",
    "Trade",
    "FDI",
    "Forex",
    "Commodity"
]

channel_map = {ch: i for i, ch in enumerate(economic_channels)}

NUM_SANCTION_TYPES = len(sanction_type_map)
NUM_CHANNELS = len(economic_channels)


In [3]:
# ---------------- NLP SANCTION MODEL ----------------

class SanctionNLPModel(nn.Module):
    def __init__(self, text_encoder, hidden_dim):
        super().__init__()
        self.encoder = text_encoder
        self.sanction_head = nn.Linear(hidden_dim, NUM_SANCTION_TYPES)
        self.channel_head = nn.Linear(hidden_dim, NUM_CHANNELS)

    def forward(self, input_ids, attention_mask):
        outputs = self.encoder(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        cls_emb = outputs.last_hidden_state[:, 0, :]
        sanction_logits = self.sanction_head(cls_emb)
        channel_logits = self.channel_head(cls_emb)

        return sanction_logits, channel_logits


In [4]:
# ---------------- LOSS FUNCTIONS ----------------

sanction_loss_fn = nn.CrossEntropyLoss()
channel_loss_fn = nn.BCEWithLogitsLoss()


In [5]:
WB_BASE = "https://api.worldbank.org/v2/country/IND/indicator/{indicator}"

WB_INDICATORS = {
    "gdp_growth": "NY.GDP.MKTP.KD.ZG",
    "cpi": "FP.CPI.TOTL.ZG",
    "exports": "NE.EXP.GNFS.CD",
    "imports": "NE.IMP.GNFS.CD",
    "fdi": "BX.KLT.DINV.CD.WD",
    "reserves": "FI.RES.TOTL.CD",
}
from datetime import datetime, timezone

def fetch_worldbank(indicator, start_year=2005, end_year=None):
    if end_year is None:
        end_year = datetime.now(timezone.utc).year

    url = WB_BASE.format(indicator=indicator)
    params = {
        "format": "json",
        "per_page": 2000,
        "date": f"{start_year}:{end_year}",
    }

    r = safe_get(url, params=params)
    if r is None:
        print(f"Failed WB fetch for {indicator}")
        return pd.DataFrame(columns=["year", indicator])

    j = r.json()
    if not isinstance(j, list) or len(j) < 2:
        return pd.DataFrame(columns=["year", indicator])

    rows = []
    for d in j[1]:
        if d["value"] is not None:
            rows.append({
                "year": int(d["date"]),
                indicator: float(d["value"])
            })

    return pd.DataFrame(rows)

In [6]:
def fetch_usdinr(start_year=2005, end_year=None):
    if end_year is None:
        end_year = datetime.now(timezone.utc).year

    url = "https://api.exchangerate.host/timeseries"
    params = {
        "start_date": f"{start_year}-01-01",
        "end_date": f"{end_year}-12-31",
        "base": "USD",
        "symbols": "INR"
    }

    r = safe_get(url, params=params)
    if r is None:
        print("⚠️ FX API failed — returning empty FX data")
        return pd.DataFrame(columns=["year", "fx"])

    data = r.json()
    rates = data.get("rates", {})

    rows = []
    for date_str, val in rates.items():
        if isinstance(val, dict) and "INR" in val:
            rows.append({
                "year": pd.to_datetime(date_str).year,
                "fx": float(val["INR"])
            })

    # 🔴 CRITICAL FIX: handle empty rows
    if not rows:
        print("⚠️ FX API returned no usable data")
        return pd.DataFrame(columns=["year", "fx"])

    df = pd.DataFrame(rows)

    return (
        df.groupby("year", as_index=False)["fx"]
          .mean()
    )


In [7]:
import requests
from time import sleep

def safe_get(url, params=None, retries=3, timeout=20, sleep_sec=2):
    """
    Safe HTTP GET with retries to avoid API timeouts.
    Returns response or None if all retries fail.
    """
    for attempt in range(1, retries + 1):
        try:
            r = requests.get(url, params=params, timeout=timeout)
            r.raise_for_status()
            return r
        except Exception as e:
            print(f"[attempt {attempt}] API error: {e}")
            if attempt == retries:
                return None
            sleep(sleep_sec)


In [8]:
def build_macro_df(start_year=1995):
    dfs = []
    for name, code in WB_INDICATORS.items():
        print(f"Fetching {name}")
        df = fetch_worldbank(code, start_year)
        df = df.rename(columns={code: name})
        dfs.append(df)

    macro_df = dfs[0]
    for df in dfs[1:]:
        macro_df = macro_df.merge(df, on="year", how="outer")

    macro_df["trade"] = macro_df["exports"] + macro_df["imports"]

    fx_df = fetch_usdinr(start_year)
    macro_df = macro_df.merge(fx_df, on="year", how="left")

    return macro_df.sort_values("year").reset_index(drop=True)


In [ ]:
import tweepy

BEARER_TOKEN ="AAAAAAAAAAAAAAAAAAAAAISv6wEAAAAACbLiuBISRVcYuHaRsmmM8jv0yZw%3D4qMK2s2TsVWFACYlVLmvKci2fK0PfVNKGRNzrxp72UKqfRzbxh"

client = tweepy.Client(
    bearer_token=BEARER_TOKEN,
    wait_on_rate_limit=True
)


In [ ]:
usernames = [
    "realDonaldTrump", # US President
    "narendramodi",    # India PM
    "RishiSunak",      # UK PM
    "EmmanuelMacron",  # France President
    "KremlinRussia_E"  # Russia
]

def get_tweets_by_user(username, max_results=10):
    user = client.get_user(username=username)
    user_id = user.data.id

    tweets = client.get_users_tweets(
        id=user_id,
        max_results=max_results,
        tweet_fields=["created_at", "lang"]
    )

    return [tweet.text for tweet in tweets.data] if tweets.data else []


In [ ]:
import time

user_cache = {}

def get_user_id(username):
    if username in user_cache:
        return user_cache[username]

    user = client.get_user(username=username)
    time.sleep(1)
    user_cache[username] = user.data.id
    return user_cache[username]

def get_tweets_by_user(username, max_results=10):
    user_id = get_user_id(username)

    tweets = client.get_users_tweets(
        id=user_id,
        max_results=max_results,
        tweet_fields=["created_at", "lang"]
    )
    time.sleep(1)

    return [tweet.text for tweet in tweets.data] if tweets.data else []


In [ ]:
tweets = []

for user in usernames:
    try:
        tweets.extend(get_tweets_by_user(user, max_results=5))
    except Exception as e:
        print(f"Error fetching from {user}: {e}")

len(tweets)

In [ ]:
def decode_nlp_output(sanction_logits, channel_logits):
    sanction_pred = torch.argmax(sanction_logits, dim=1)

    channel_probs = torch.sigmoid(channel_logits)
    channel_pred = (channel_probs > 0.5).int()

    sanction_type = list(sanction_type_map.keys())[sanction_pred.item()]
    channels = [
        economic_channels[i]
        for i in range(NUM_CHANNELS)
        if channel_pred[0][i] == 1
    ]

    impact_strength = channel_probs.mean().item()

    return {
        "sanction_type": sanction_type,
        "economic_channels": channels,
        "impact_strength": round(impact_strength, 3)
    }


In [9]:
COMTRADE_URL = "https://comtradeapi.worldbank.org/v1/get/HS"

def fetch_trade(year, top_k=10):
    """
    Fetch India total trade (imports + exports) with top partners for a year.
    Returns DataFrame: partner, value
    """
    rows = []

    for rg in [1, 2]:  # 1=imports, 2=exports
        params = {
            "reporterCode": 356,   # India
            "partnerCode": 0,
            "year": year,
            "rgCode": rg,
            "cmdCode": "TOTAL",
            "freq": "A",
            "format": "json"
        }

        r = safe_get(COMTRADE_URL, params=params)
        if r is None:
            print(f"⚠️ Comtrade failed for {year}, rg={rg}")
            continue

        data = r.json().get("data", [])
        for d in data:
            rows.append({
                "partner": d.get("ptTitle"),
                "value": float(d.get("TradeValue", 0.0))
            })

        sleep(1)  # respect rate limits

    if not rows:
        return pd.DataFrame(columns=["partner", "value"])

    df = pd.DataFrame(rows)

    return (
        df.groupby("partner", as_index=False)["value"]
          .sum()
          .sort_values("value", ascending=False)
          .head(top_k)
          .reset_index(drop=True)
    )

from torch_geometric.data import Data
import numpy as np
import torch

def build_graph(year, macro_df, india_index=0, num_partners=5):
    """
    Fallback trade graph when Comtrade is unavailable.
    Creates synthetic partner nodes weighted by trade volume.
    """

    row = macro_df[macro_df.year == year]
    if row.empty:
        return None

    # Scalars (IMPORTANT: use .item())
    exports = row["exports"].item()
    imports = row["imports"].item()

    # Countries
    partners = [f"Partner_{i}" for i in range(num_partners)]
    countries = ["India"] + partners
    num_nodes = len(countries)

    # Node features (all macro features except year)
    feature_cols = macro_df.columns.drop("year")
    F = len(feature_cols)

    # Extract macro features as PURE FLOATS
    feat_vals = row[feature_cols].astype(float).values[0]
    feat_vals = np.nan_to_num(feat_vals, nan=0.0)

    # Create node feature matrix
    x = torch.tensor(
        np.repeat(feat_vals.reshape(1, F), num_nodes, axis=0),
        dtype=torch.float32
    )

    # Trade-based edges
    total_trade = exports + imports
    weights = torch.linspace(1.0, 0.3, steps=num_partners)
    weights = weights / weights.sum() * total_trade

    src, dst, w = [], [], []

    for i in range(num_partners):
        j = i + 1
        src += [india_index, j]
        dst += [j, india_index]
        w += [weights[i].item(), weights[i].item()]

    edge_index = torch.tensor([src, dst], dtype=torch.long)
    edge_weight = torch.tensor(w, dtype=torch.float32)

    if edge_weight.numel() > 0:
        edge_weight /= edge_weight.max()

    g = Data(x=x, edge_index=edge_index, edge_weight=edge_weight)
    g.year = year

    return g

In [10]:
def calculate_recovery_duration(df, indicator="gdp_growth", window=3, max_look=5):
    """
    Calculates years to recovery for each year in the dataframe.
    """
    durations = []
    vals = df[indicator].values
    
    for i in range(len(vals)):
        # Calculate pre-event baseline (average of previous 'window' years)
        baseline = np.mean(vals[max(0, i-window):i]) if i > 0 else vals[i]
        
        # We only care if the current year is a 'drop' year (potential sanction impact)
        if vals[i] < baseline - 0.5: # 0.5% threshold drop
            recovered_in = max_look
            for lookahead in range(1, max_look + 1):
                if i + lookahead < len(vals):
                    if vals[i + lookahead] >= baseline:
                        recovered_in = lookahead
                        break
            durations.append(float(recovered_in))
        else:
            durations.append(0.0) # No active duration/recovery needed
            
    return torch.tensor(durations, dtype=torch.float32).unsqueeze(1)

# UPDATE your build_labels function like this:
def build_labels(macro_df):
    labels = {}
    # ... your existing gdp, cpi, fx, trade, fdi, res code ...
    
    # 🔴 INTEGRATED DURATION LOGIC
    labels["duration"] = calculate_recovery_duration(macro_df, indicator="gdp_growth")
    
    # ... your existing score calculation ...
    return labels

In [11]:
def make_targets(macro_df: pd.DataFrame, year: int, countries: List[str]) -> np.ndarray:
    # Set index to allow easy lookup
    macro_indexed = macro_df.set_index(["country", "year"])
    y_cols = ["gdp_growth", "cpi", "fx", "trade", "fdi", "reserves"]

    Y = []
    for c in countries:
        try:
            row = macro_indexed.loc[(c, year)]
            vals = row[y_cols].values.astype(float)
            
            # 🔴 CALCULATE DURATION DYNAMICALLY
            # Look at the country's history in the full macro_df
            c_history = macro_df[macro_df["country"] == c].sort_values("year")
            c_history = c_history[c_history["year"] <= year].tail(5) # Look at recent window
            
            baseline = c_history["gdp_growth"].mean()
            current_gdp = row["gdp_growth"]
            
            # Simple heuristic: if GDP is 1% below baseline, predict it takes 2-4 years
            if current_gdp < baseline - 1.0:
                duration = 3.5 # Estimated impact length in years
            else:
                duration = 0.0
                
        except KeyError:
            vals = np.zeros(len(y_cols))
            duration = 0.0

        score = float(np.mean(np.abs(vals)))
        Y.append(np.concatenate([vals, [score, duration]]))

    return np.vstack(Y)

In [12]:
def build_labels(macro_df):
    labels = {}

    labels["gdp"] = torch.tensor(
        macro_df["gdp_growth"].astype(float).to_numpy(),
        dtype=torch.float32
    ).unsqueeze(1)

    labels["cpi"] = torch.tensor(
        macro_df["cpi"].astype(float).to_numpy(),
        dtype=torch.float32
    ).unsqueeze(1)

    labels["fx"] = torch.tensor(
        macro_df["fx"].astype(float).to_numpy(),
        dtype=torch.float32
    ).unsqueeze(1)

    labels["trade"] = torch.tensor(
        macro_df["trade"].astype(float).to_numpy(),
        dtype=torch.float32
    ).unsqueeze(1)

    labels["fdi"] = torch.tensor(
        macro_df["fdi"].astype(float).to_numpy(),
        dtype=torch.float32
    ).unsqueeze(1)

    labels["res"] = torch.tensor(
        macro_df["reserves"].astype(float).to_numpy(),
        dtype=torch.float32
    ).unsqueeze(1)

    labels["score"] = (
        labels["gdp"] * 0.3 +
        labels["trade"] * 0.2 +
        labels["fdi"] * 0.2 -
        labels["cpi"] * 0.2 -
        labels["fx"] * 0.1
    )

    labels["duration"] = torch.ones_like(labels["gdp"])

    return labels


In [13]:
# -------------------------------
# Force all macro features to numeric (MANDATORY)
# -------------------------------
macro_df = build_macro_df(2005)

for col in macro_df.columns:
    if col != "year":
        macro_df[col] = pd.to_numeric(macro_df[col], errors="coerce")

# Handle remaining NaNs safely
macro_df = macro_df.fillna(0.0)

print(macro_df.dtypes)


Fetching gdp_growth
Fetching cpi
Fetching exports
Fetching imports
Fetching fdi
Fetching reserves
⚠️ FX API returned no usable data
year            int64
gdp_growth    float64
cpi           float64
exports       float64
imports       float64
fdi           float64
reserves      float64
trade         float64
fx            float64
dtype: object


In [14]:
# -------------------------------------------------
# 1. Build macroeconomic dataframe
# -------------------------------------------------
macro_df = build_macro_df(2005)

# -------------------------------------------------
# 2. Build training labels (dict of tensors)
# -------------------------------------------------
y_targets = build_labels(macro_df)
y_targets["fx"] = torch.nan_to_num(y_targets["fx"], nan=0.0)
y_targets["score"] = (
    y_targets["gdp"] * 0.3 +
    y_targets["trade"] * 0.2 +
    y_targets["fdi"] * 0.2 -
    y_targets["cpi"] * 0.2 -
    y_targets["fx"] * 0.1
)
# -------------------------------------------------
# 3. Build yearly sanction graphs
# -------------------------------------------------
graphs = []

for y in macro_df["year"]:
    g = build_graph(y, macro_df)
    if g is not None:
        graphs.append(g)

# -------------------------------------------------
# 4. Sanity checks
# -------------------------------------------------
print("Graphs:", len(graphs))

for k, v in y_targets.items():
    print(f"{k}: {v.shape}")


Fetching gdp_growth
Fetching cpi
Fetching exports
Fetching imports
Fetching fdi
Fetching reserves
⚠️ FX API returned no usable data
Graphs: 20
gdp: torch.Size([20, 1])
cpi: torch.Size([20, 1])
fx: torch.Size([20, 1])
trade: torch.Size([20, 1])
fdi: torch.Size([20, 1])
res: torch.Size([20, 1])
score: torch.Size([20, 1])
duration: torch.Size([20, 1])


In [15]:
import torch
import torch.nn as nn

# -------------------------------
# 1. Device
# -------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# -------------------------------
# 2. Move graphs to device
# -------------------------------
graphs_dev = [g.to(device) for g in graphs]

# -------------------------------
# 3. Prepare & normalize targets
# -------------------------------
y_targets = {k: v.to(device).float() for k, v in y_targets.items()}

# Z-score normalization (VERY IMPORTANT)
y_mean, y_std = {}, {}
for k in y_targets:
    y_targets[k] = torch.nan_to_num(y_targets[k], nan=0.0)

    y_mean[k] = y_targets[k].mean()
    y_std[k] = y_targets[k].std()

    if y_std[k] < 1e-5:
        print(f"⚠️ Skipping normalization for {k} (std≈0)")
        y_targets[k] = y_targets[k] - y_mean[k]
    else:
        y_targets[k] = (y_targets[k] - y_mean[k]) / (y_std[k] + 1e-6)

print("Targets normalized.")

# -------------------------------
# 4. Initialize model
# -------------------------------
model = SanctionImpactGNN(
    in_dim=graphs_dev[0].x.shape[1],
    india_index=0
).to(device)


# -------------------------------
# 5. Optimizer & loss
# -------------------------------
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=1e-3,
    weight_decay=1e-4
)

loss_fn = nn.MSELoss()

# Optional: economic importance weights
loss_weights = {
    "gdp": 2.0,
    "cpi": 2.0,
    "fx": 1.5,
    "trade": 1.0,
    "fdi": 1.0,
    "res": 1.0,
    "score": 1.5,
    "duration": 1.0
}

# -------------------------------
# 6. Training loop
# -------------------------------
EPOCHS = 300
model.train()

for epoch in range(EPOCHS):
    optimizer.zero_grad()

    # Forward pass
    preds = model(graphs_dev)   # dict: key -> [T, 1]

    # Multi-task weighted loss
    loss = 0.0
    for k in y_targets:
        loss += loss_weights[k] * loss_fn(preds[k], y_targets[k])

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Logging
    if epoch % 25 == 0 or epoch == EPOCHS - 1:
        print(f"Epoch {epoch:03d} | Loss: {loss.item():.4f}")

print("Training complete.")


Using device: cpu
⚠️ Skipping normalization for fx (std≈0)
⚠️ Skipping normalization for duration (std≈0)
Targets normalized.
Epoch 000 | Loss: 11.9930
Epoch 025 | Loss: 8.9024
Epoch 050 | Loss: 8.6774
Epoch 075 | Loss: 8.2111
Epoch 100 | Loss: 8.3894
Epoch 125 | Loss: 7.2849
Epoch 150 | Loss: 8.0230
Epoch 175 | Loss: 8.4455
Epoch 200 | Loss: 8.0204
Epoch 225 | Loss: 7.6073
Epoch 250 | Loss: 7.4550
Epoch 275 | Loss: 7.5819
Epoch 299 | Loss: 7.6245
Training complete.


In [16]:
for k, v in y_targets.items():
    print(
        k,
        "mean:", v.mean().item(),
        "std:", v.std().item(),
        "nan count:", torch.isnan(v).sum().item()
    )

gdp mean: 1.668930025289228e-07 std: 0.9999997019767761 nan count: 0
cpi mean: -5.9604645663569045e-09 std: 0.9999996423721313 nan count: 0
fx mean: 0.0 std: 0.0 nan count: 0
trade mean: -6.407499597571586e-08 std: 1.0 nan count: 0
fdi mean: 2.9802322387695312e-08 std: 0.9999999403953552 nan count: 0
res mean: -2.3841858265427618e-08 std: 1.0 nan count: 0
score mean: 3.0249356086642365e-07 std: 1.0 nan count: 0
duration mean: 0.0 std: 0.0 nan count: 0


In [17]:
model.eval()
with torch.no_grad():
    preds_norm = model(graphs_dev)

preds_real = {}
for k in preds_norm:
    preds_real[k] = preds_norm[k] * y_std[k] + y_mean[k]

In [19]:
import torch
import ipywidgets as widgets
from IPython.display import display, clear_output

# -------- CONFIG --------
india_index = 0        # same as model
base_feature_dim = 10  # macro features already in data.x
policy_dim = 7         # number of NLP policy signals

# -------- WIDGETS --------
severity = widgets.FloatSlider(
    value=0.5, min=0.0, max=1.0, step=0.05,
    description="Severity"
)

financial = widgets.Checkbox(description="Financial")
trade = widgets.Checkbox(description="Trade")
technology = widgets.Checkbox(description="Technology")
energy = widgets.Checkbox(description="Energy")

issuer_strength = widgets.FloatSlider(
    value=0.7, min=0.0, max=1.0, step=0.05,
    description="Issuer Power"
)

binding = widgets.Checkbox(description="Legally Binding")

apply_btn = widgets.Button(
    description="Apply Policy Signal",
    button_style="success"
)

output = widgets.Output()

# -------- CALLBACK --------
def apply_policy(_):
    with output:
        clear_output()

        policy_vector = torch.tensor([
            severity.value,
            float(financial.value),
            float(trade.value),
            float(technology.value),
            float(energy.value),
            issuer_strength.value,
            float(binding.value)
        ], dtype=torch.float32)

        print("📌 Policy Signal Vector:")
        print(policy_vector)
        print("\nShape:", policy_vector.shape)

        print("\nInterpretation:")
        print(f"Severity: {policy_vector[0]:.2f}")
        print(f"Types: Financial={financial.value}, Trade={trade.value}, "
              f"Tech={technology.value}, Energy={energy.value}")
        print(f"Issuer strength: {policy_vector[5]:.2f}")
        print(f"Binding: {binding.value}")

apply_btn.on_click(apply_policy)

# -------- DISPLAY --------
display(
    widgets.VBox([
        severity,
        widgets.HBox([financial, trade, technology, energy]),
        issuer_strength,
        binding,
        apply_btn,
        output
    ])
)